In [1]:
"""Methods for polynomials."""


import numpy as np


def briot_ruffini(a, root):
    """Divide a polynomial by another polynomial.

    The format is: P(x) = Q(x) * (x-root) + rest.

    Args:
        a: an array containing the coefficients of the input polynomial.
        root: one of the polynomial roots.

    Returns:
        b: an array containing the coefficients of the output polynomial.
        rest: polynomial division Rest.
    """
    n = a.size - 1
    b = np.zeros(n)

    b[0] = a[0]

    for i in range(1, n):
        b[i] = b[i - 1] * root + a[i]

    rest = b[n - 1] * root + a[n]

    return [b, rest]


def newton_divided_difference(x, y):
    """Find the coefficients of Newton's divided difference.

    Also, find Newton's polynomial.

    Args:
        x: an array containing x values.
        y: an array containing y values.

    Returns:
        f: an array containing Newton's divided difference coefficients.
    """
    n = x.size
    q = np.zeros((n, n - 1))

    # Insert 'y' in the first column of the matrix 'q'
    q = np.concatenate((y[:, None], q), axis=1)

    for i in range(1, n):
        for j in range(1, i + 1):
            q[i, j] = (q[i, j - 1] - q[i - 1, j - 1]) / (x[i] - x[i - j])

    # Copy the diagonal values of the matrix q to the vector f
    f = np.zeros(n)
    for i in range(0, n):
        f[i] = q[i, i]

    # Prints the polynomial
    print("The polynomial is:")
    print("p(x)={:+.4f}".format(f[0]), end="")
    for i in range(1, n):
        print("{:+.4f}".format(f[i]), end="")
        for j in range(1, i + 1):
            print("(x{:+.4f})".format(x[j] * -1), end="")
    print("")

    return [f]


In [4]:
briot_ruffini(np.array([4,5]),5)

[array([4.]), 25.0]

In [5]:
"""Methods for solutions of equations."""

import math


def bisection(f, a, b, tol, iter_max):
    """Calculate the root of an equation by the Bisection method.

    Args:
        f: function f(x).
        a: lower limit.
        b: upper limit.
        tol: tolerance.
        iter_max: maximum number of iterations.

    Returns:
        root: root value.
        iter: used iterations.
        converged: found the root.
    """
    fa = f(a)
    fb = f(b)

    if fa * fb > 0:
        raise Exception("The function does not change signal at \
              the ends of the given interval.")

    delta_x = math.fabs(b - a) / 2

    x = 0
    converged = False
    i = 0
    for i in range(0, iter_max + 1):
        x = (a + b) / 2
        fx = f(x)

        print("i: {:03d}\t x: {:+.4f}\t fx: {:+.4f}\t dx: {:+.4f}\n"
              .format(i, x, fx, delta_x), end="")

        if delta_x <= tol and math.fabs(fx) <= tol:
            converged = True
            break

        if fa * fx > 0:
            a = x
            fa = fx
        else:
            b = x

        delta_x = delta_x / 2
    else:
        print("Warning: The method did not converge.")

    root = x
    return [root, i, converged]


def newton(f, df, x0, tol, iter_max):
    """Calculate the root of an equation by the Newton method.

    Args:
        f: function f(x).
        df: derivative of function f(x).
        x0: initial guess.
        tol: tolerance.
        iter_max: maximum number of iterations.

    Returns:
        root: root value.
        iter: used iterations.
        converged: found the root.
    """
    x = x0
    fx = f(x)
    dfx = df(x)

    converged = False
    print("iter: 0 x: {:.4f}\t dfx: {:.4f}\t fx: {:.4f}\n"
          .format(x, dfx, fx), end="")

    i = 0
    for i in range(1, iter_max + 1):
        delta_x = -fx / dfx
        x += delta_x
        fx = f(x)
        dfx = df(x)

        print("i:{:03d}\t x: {:.4f}\t dfx: {:.4f}\t fx: {:.4f}\t dx: {:.4f}\n"
              .format(i, x, dfx, fx, delta_x), end="")

        if math.fabs(delta_x) <= tol and math.fabs(fx) <= tol or dfx == 0:
            converged = True
            break
    else:
        print("Warning: The method did not converge.")

    root = x
    return [root, i, converged]


def secant(f, a, b, tol, iter_max):
    """Calculate the root of an equation by the Secant method.

    Args:
        f: function f(x).
        a: lower limit.
        b: upper limit.
        tol: tolerance.
        iter_max: maximum number of iterations.

    Returns:
        root: root value.
        iter: used iterations.
        converged: found the root.
    """
    fa = f(a)
    fb = f(b)

    if fb - fa == 0:
        raise Exception("f(b)-f(a) must be nonzero.")

    if b - a == 0:
        raise Exception("b-a must be nonzero.")

    if math.fabs(fa) < math.fabs(fb):
        a, b = b, a
        fa, fb = fb, fa

    x = b
    fx = fb

    converged = False
    i = 0
    for i in range(0, iter_max + 1):
        delta_x = -fx / (fb - fa) * (b - a)
        x += delta_x
        fx = f(x)

        print("i: {:03d}\t x: {:+.4f}\t fx: {:+.4f}\t dx: {:+.4f}\n"
              .format(i, x, fx, delta_x), end="")

        if math.fabs(delta_x) <= tol and math.fabs(fx) <= tol:
            converged = True
            break

        a, b = b, x
        fa, fb = fb, fx
    else:
        print("Warning: The method did not converge.")

    root = x
    return [root, i, converged]


In [6]:
"""Methods for ordinary differential equations."""

import numpy as np


def euler(f, a, b, n, ya):
    """Calculate the solution of the initial-value problem (IVP).

    Solve the IVP from the Euler method.

    Args:
        f: function f(x).
        a: the initial point.
        b: the final point.
        n: number of intervals.
        ya: initial value.

    Returns:
        vx: an array containing x values.
        vy: an array containing y values (solution of IVP).
    """
    vx = np.zeros(n)
    vy = np.zeros(n)

    h = (b - a) / n
    x = a
    y = ya

    vx[0] = x
    vy[0] = y

    fxy = f(x, y)
    print("i: {:03d}\t x: {:.4f}\t y: {:.4f}\n".format(0, x, y), end="")

    for i in range(0, n):
        x = a + (i + 1) * h
        y += h * fxy

        fxy = f(x, y)
        print("i: {:03d}\t x: {:.4f}\t y: {:.4f}\n"
              .format(i + 1, x, y), end="")
        vx[i] = x
        vy[i] = y

    return [vx, vy]


def taylor2(f, df1, a, b, n, ya):
    """Calculate the solution of the initial-value problem (IVP).

    Solve the IVP from the Taylor (Order Two) method.

    Args:
        f: function f(x).
        df1: 1's derivative of function f(x).
        a: the initial point.
        b: the final point.
        n: number of intervals.
        ya: initial value.

    Returns:
        vx: an array containing x values.
        vy: an array containing y values (solution of IVP).
    """
    vx = np.zeros(n)
    vy = np.zeros(n)

    h = (b - a) / n
    x = a
    y = ya

    vx[0] = x
    vy[0] = y

    print("i: {:03d}\t x: {:.4f}\t y: {:.4f}\n".format(0, x, y), end="")

    for i in range(0, n):
        y += h * (f(x, y) + 0.5 * h * df1(x, y))
        x = a + (i + 1) * h

        print(
            "i: {:03d}\t x: {:.4f}\t y: {:.4f}\n".format(
                i + 1, x, y), end="")
        vx[i] = x
        vy[i] = y

    return [vx, vy]


def taylor4(f, df1, df2, df3, a, b, n, ya):
    """Calculate the solution of the initial-value problem (IVP).

    Solve the IVP from the Taylor (Order Four) method.

    Args:
        f: function f(x).
        df1: 1's derivative of function f(x).
        df2: 2's derivative of function f(x).
        df3: 3's derivative of function f(x).
        a: the initial point.
        b: the final point.
        n: number of intervals.
        ya: initial value.

    Returns:
        vx: an array containing x values.
        vy: an array containing y values (solution of IVP).
    """
    vx = np.zeros(n)
    vy = np.zeros(n)

    h = (b - a) / n
    x = a
    y = ya

    vx[0] = x
    vy[0] = y

    print("i: {:03d}\t x: {:.4f}\t y: {:.4f}\n".format(0, x, y), end="")

    for i in range(0, n):
        y += h * (f(x, y) + 0.5 * h * df1(x, y) + (h ** 2 / 6) * df2(x, y) +
                  (h ** 3 / 24) * df3(x, y))
        x = a + (i + 1) * h

        print(
            "i: {:03d}\t x: {:.4f}\t y: {:.4f}\n".format(
                i + 1, x, y), end="")
        vx[i] = x
        vy[i] = y

    return [vx, vy]


def rk4(f, a, b, n, ya):
    """Calculate the solution of the initial-value problem (IVP).

    Solve the IVP from the Runge-Kutta (Order Four) method.

    Args:
        f: function f(x).
        a: the initial point.
        b: the final point.
        n: number of intervals.
        ya: initial value.

    Returns:
        vx: an array containing x values.
        vy: an array containing y values (solution of IVP).
    """
    vx = np.zeros(n)
    vy = np.zeros(n)

    h = (b - a) / n
    x = a
    y = ya

    k = np.zeros(4)

    vx[0] = x
    vy[0] = y

    print("i: {:03d}\t x: {:.4f}\t y: {:.4f}\n".format(0, x, y), end="")

    for i in range(0, n):
        k[0] = h * f(x, y)
        k[1] = h * f(x + h / 2, y + k[0] / 2)
        k[2] = h * f(x + h / 2, y + k[1] / 2)
        k[3] = h * f(x + h, y + k[2])

        x = a + (i + 1) * h
        y += (k[0] + 2 * k[1] + 2 * k[2] + k[3]) / 6

        print(
            "i: {:03d}\t x: {:.4f}\t y: {:.4f}\n".format(
                i + 1, x, y), end="")
        vx[i] = x
        vy[i] = y

    return [vx, vy]


def rk4_system(f, a, b, n, ya):
    """Calculate the solution of systems of differential equations.

    Solve from Runge-Kutta (Order Four) method.

    Args:
        f: an array of functions f(x).
        a: the initial point.
        b: the final point.
        n: number of intervals.
        ya: an array of initial values.

    Returns:
        vx: an array containing x values.
        vy: an array containing y values (solution of IVP).
    """
    m = len(f)

    k = [np.zeros(m), np.zeros(m), np.zeros(m), np.zeros(m)]

    vx = np.zeros(n + 1)
    vy = np.zeros((m, n + 1))

    h = (b - a) / n

    x = a
    y = ya

    vx[0] = x
    vy[:, 0] = y

    for i in range(0, n):

        for j in range(0, m):
            k[0][j] = h * f[j](x, y)

        for j in range(0, m):
            k[1][j] = h * f[j](x + h / 2, y + k[0] / 2)

        for j in range(0, m):
            k[2][j] = h * f[j](x + h / 2, y + k[1] / 2)

        for j in range(0, m):
            k[3][j] = h * f[j](x + h, y + k[2])

        x = a + i * h
        y = y + (k[0] + 2 * k[1] + 2 * k[2] + k[3]) / 6

        vx[i + 1] = x
        vy[:, i + 1] = y

    return [vx, vy]


In [7]:
"""Methods for Linear Systems."""

import numpy as np


def jacobi(a, b, x0, tol, iter_max):
    """Jacobi method: solve Ax = b given an initial approximation x0.

    Args:
        a: matrix A from system Ax=b.
        b: an array containing b values.
        x0: initial approximation of the solution.
        tol: tolerance.
        iter_max: maximum number of iterations.

    Returns:
        x: solution of linear the system.
        iter: used iterations.
    """
    # D and M matrices
    d = np.diag(np.diag(a))
    m = a - d

    # Iterative process
    i = 1
    x = None
    for i in range(1, iter_max + 1):
        x = np.linalg.solve(d, (b - np.dot(m, x0)))

        if np.linalg.norm(x - x0, np.inf) / np.linalg.norm(x, np.inf) <= tol:
            break
        x0 = x.copy()

    return [x, i]


def gauss_seidel(a, b, x0, tol, iter_max):
    """Gauss-Seidel method: solve Ax = b given an initial approximation x0.

    Args:
        a: matrix A from system Ax=b.
        b: an array containing b values.
        x0: initial approximation of the solution.
        tol: tolerance.
        iter_max: maximum number of iterations.

    Returns:
        x: solution of linear the system.
        iter: used iterations.
    """
    # L and U matrices
    lower = np.tril(a)
    upper = a - lower

    # Iterative process
    i = 1
    x = None
    for i in range(1, iter_max + 1):
        x = np.linalg.solve(lower, (b - np.dot(upper, x0)))

        if np.linalg.norm(x - x0, np.inf) / np.linalg.norm(x, np.inf) <= tol:
            break
        x0 = x.copy()

    return [x, i]


In [8]:
"""Iterative Methods for Linear Systems."""

import math
import numpy as np


def backward_substitution(upper, d):
    """Solve the upper linear system ux=d.

    Args:
        upper: upper triangular matrix.
        d: an array containing d values.

    Returns:
        x: solution of linear the system.
    """
    [n, m] = upper.shape
    b = d.astype(float)

    if n != m:
        raise Exception("'upper' must be a square matrix.")

    x = np.zeros(n)

    for i in range(n - 1, -1, -1):
        if upper[i, i] == 0:
            raise Exception("Matrix 'upper' is singular.")

        x[i] = b[i] / upper[i, i]
        b[0:i] = b[0:i] - upper[0:i, i] * x[i]

    return [x]


def forward_substitution(lower, c):
    """Solve the lower linear system lx=c.

    Args:
        lower: lower triangular matrix.
        c: an array containing c values.

    Returns:
        x: solution of linear the system.
    """
    [n, m] = lower.shape
    b = c.astype(float)

    if n != m:
        raise Exception("'lower' must be a square matrix.")

    x = np.zeros(n)

    for i in range(0, n):
        if lower[i, i] == 0:
            raise Exception("Matrix 'lower' is singular.")

        x[i] = b[i] / lower[i, i]
        b[i + 1:n] = b[i + 1:n] - lower[i + 1:n, i] * x[i]

    return [x]


def gauss_elimination_pp(a, b):
    """Gaussian Elimination with Partial Pivoting.

    Calculate the upper triangular matrix from linear system Ax=b (make a row
    reduction).

    Args:
        a: matrix A from system Ax=b.
        b: an array containing b values.

    Returns:
        a: augmented upper triangular matrix.
    """
    [n, m] = a.shape

    if n != m:
        raise Exception("'a' must be a square matrix.")

    # Produces the augmented matrix
    a = np.concatenate((a, b[:, None]), axis=1).astype(float)

    # Elimination process starts
    for i in range(0, n - 1):
        p = i

        # Comparison to select the pivot
        for j in range(i + 1, n):
            if math.fabs(a[j, i]) > math.fabs(a[i, i]):
                # Swap rows
                a[[i, j]] = a[[j, i]]

        # Checking for nullity of the pivots
        while p < n and a[p, i] == 0:
            p += 1

        if p == n:
            print("Info: No unique solution.")
        else:
            if p != i:
                # Swap rows
                a[[i, p]] = a[[p, i]]

        for j in range(i + 1, n):
            a[j, :] = a[j, :] - a[i, :] * (a[j, i] / a[i, i])

    # Checking for nonzero of last entry
    if a[n - 1, n - 1] == 0:
        print("Info: No unique solution.")

    return [a]


In [9]:
"""Methods for Interpolation."""

import numpy as np


def lagrange(x, y, x_int):
    """Interpolates a value using the 'Lagrange polynomial'.

    Args:
        x: an array containing x values.
        y: an array containing y values.
        x_int: value to interpolate.

    Returns:
        y_int: interpolated value.
    """
    n = x.size
    y_int = 0

    for i in range(0, n):
        p = y[i]
        for j in range(0, n):
            if i != j:
                p = p * (x_int - x[j]) / (x[i] - x[j])
        y_int = y_int + p

    return [y_int]


def neville(x, y, x_int):
    """Interpolates a value using the 'Neville polynomial'.

    Args:
        x: an array containing x values.
        y: an array containing y values.
        x_int: value to interpolate.

    Returns:
        y_int: interpolated value.
        q: coefficients matrix.
    """
    n = x.size
    q = np.zeros((n, n - 1))

    # Insert 'y' in the first column of the matrix 'q'
    q = np.concatenate((y[:, None], q), axis=1)

    for i in range(1, n):
        for j in range(1, i + 1):
            q[i, j] = ((x_int - x[i - j]) * q[i, j - 1] -
                       (x_int - x[i]) * q[i - 1, j - 1]) / (x[i] - x[i - j])

    y_int = q[n - 1, n - 1]
    return [y_int, q]


In [10]:
"""Methods for numerical integration."""


def composite_simpson(f, b, a, n):
    """Calculate the integral from 1/3 Simpson's Rule.

    Args:
        f: function f(x).
        a: the initial point.
        b: the final point.
        n: number of intervals.

    Returns:
        xi: integral value.
    """
    h = (b - a) / n

    sum_odd = 0
    sum_even = 0

    for i in range(0, n - 1):
        x = a + (i + 1) * h
        if (i + 1) % 2 == 0:
            sum_even += f(x)
        else:
            sum_odd += f(x)

    xi = h / 3 * (f(a) + 2 * sum_even + 4 * sum_odd + f(b))
    return [xi]


def composite_trapezoidal(f, b, a, n):
    """Calculate the integral from the Trapezoidal Rule.

    Args:
        f: function f(x).
        a: the initial point.
        b: the final point.
        n: number of intervals.

    Returns:
        xi: integral value.
    """
    h = (b - a) / n

    sum_x = 0

    for i in range(0, n - 1):
        x = a + (i + 1) * h
        sum_x += f(x)

    xi = h / 2 * (f(a) + 2 * sum_x + f(b))
    return [xi]


def composite2_simpson(x, y):
    """Calculate the integral from 1/3 Simpson's Rule.

    Args:
        x: an array containing x values.
        y: an array containing y values.

    Returns:
        xi: integral value.
    """
    if y.size != y.size:
        raise Exception("'x' and 'y' must have same size.")

    h = x[1] - x[0]
    n = x.size

    sum_odd = 0
    sum_even = 0

    for i in range(1, n - 1):
        if (i + 1) % 2 == 0:
            sum_even += y[i]
        else:
            sum_odd += y[i]

    xi = h / 3 * (y[0] + 2 * sum_even + 4 * sum_odd + y[n - 1])
    return [xi]


def composite2_trapezoidal(x, y):
    """Calculate the integral from the Trapezoidal Rule.

    Args:
        x: an array containing x values.
        y: an array containing y values.

    Returns:
        xi: integral value.
    """
    if y.size != y.size:
        raise Exception("'x' and 'y' must have same size.")

    h = x[1] - x[0]
    n = x.size

    sum_x = 0

    for i in range(1, n - 1):
        sum_x += y[i]

    xi = h / 2 * (y[0] + 2 * sum_x + y[n - 1])
    return [xi]


In [11]:
"""Numerical differentiation."""

import numpy as np


def derivative_backward_difference(x, y):
    """Calculate the first derivative.

    All values in 'x' must be equally spaced.

    Args:
        x: an array containing x values.
        y: an array containing y values.

    Returns:
        dy: an array containing the first derivative values.
    """
    if x.size < 2 or y.size < 2:
        raise Exception("'x' and 'y' arrays must have 2 values or more.")

    if x.size != y.size:
        raise Exception("'x' and 'y' must have same size.")

    def dy_difference(h, y0, y1):
        return (y1 - y0) / h

    n = x.size
    dy = np.zeros(n)
    for i in range(0, n):
        if i == n - 1:
            hx = x[i] - x[i - 1]
            dy[i] = dy_difference(-hx, y[i], y[i - 1])
        else:
            hx = x[i + 1] - x[i]
            dy[i] = dy_difference(hx, y[i], y[i + 1])

    return [dy]


def derivative_three_point(x, y):
    """Calculate the first derivative.

    All values in 'x' must be equally spaced.

    Args:
        x: an array containing x values.
        y: an array containing y values.

    Returns:
        dy: an array containing the first derivative values.
    """
    if x.size < 3 or y.size < 3:
        raise Exception("'x' and 'y' arrays must have 3 values or more.")

    if x.size != y.size:
        raise Exception("'x' and 'y' must have same size.")

    def dy_mid(h, y0, y2):
        return (1 / (2 * h)) * (y2 - y0)

    def dy_end(h, y0, y1, y2):
        return (1 / (2 * h)) * (-3 * y0 + 4 * y1 - y2)

    hx = x[1] - x[0]
    n = x.size
    dy = np.zeros(n)
    for i in range(0, n):
        if i == 0:
            dy[i] = dy_end(hx, y[i], y[i + 1], y[i + 2])
        elif i == n - 1:
            dy[i] = dy_end(-hx, y[i], y[i - 1], y[i - 2])
        else:
            dy[i] = dy_mid(hx, y[i - 1], y[i + 1])

    return [dy]


def derivative_five_point(x, y):
    """Calculate the first derivative.

    All values in 'x' must be equally spaced.

    Args:
        x: an array containing x values.
        y: an array containing y values.

    Returns:
        dy: an array containing the first derivative values.
    """
    if x.size < 6 or y.size < 6:
        raise Exception("'x' and 'y' arrays must have 6 values or more.")

    if x.size != y.size:
        raise Exception("'x' and 'y' must have same size.")

    def dy_mid(h, y0, y1, y3, y4):
        return (1 / (12 * h)) * (y0 - 8 * y1 + 8 * y3 - y4)

    def dy_end(h, y0, y1, y2, y3, y4):
        return (1 / (12 * h)) * \
               (-25 * y0 + 48 * y1 - 36 * y2 + 16 * y3 - 3 * y4)

    hx = x[1] - x[0]
    n = x.size
    dy = np.zeros(n)
    for i in range(0, n):
        if i in (0, 1):
            dy[i] = dy_end(hx, y[i], y[i + 1], y[i + 2], y[i + 3], y[i + 4])
        elif i in (n - 1, n - 2):
            dy[i] = dy_end(-hx, y[i], y[i - 1], y[i - 2], y[i - 3], y[i - 4])
        else:
            dy[i] = dy_mid(hx, y[i - 2], y[i - 1], y[i + 1], y[i + 2])

    return [dy]
